In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Project

/content/drive/MyDrive/Project


In [ ]:
import pandas as pd
#load_data
df_wildlife = pd.read_csv('Wildlife_trade.csv')
df_species = pd.read_csv('National_Park/species.csv')
df_parks = pd.read_csv('National_Park/parks.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Origianl Data Overview

In [ ]:
df_wildlife.head()

,Year,App.,Taxon,Class,Order,Family,Genus,Importer,Exporter,Origin,Importer reported quantity,Exporter reported quantity,Term,Unit,Purpose,Source
0,2016,I,Aquila heliaca,Aves,Falconiformes,Accipitridae,Aquila,TR,NL,CZ,NaN,1.0,bodies,NaN,T,C
1,2016,I,Aquila heliaca,Aves,Falconiformes,Accipitridae,Aquila,XV,RS,RS,NaN,1.0,bodies,NaN,Q,O
2,2016,I,Haliaeetus albicilla,Aves,Falconiformes,Accipitridae,Haliaeetus,BE,NO,NaN,NaN,43.0,feathers,NaN,S,W
3,2016,I,Haliaeetus albicilla,Aves,Falconiformes,Accipitridae,Haliaeetus,BE,NO,NaN,NaN,43.0,specimens,NaN,S,W
4,2016,I,Haliaeetus albicilla,Aves,Falconiformes,Accipitridae,Haliaeetus,DK,IS,NaN,700.0,NaN,specimens,NaN,S,W


In [ ]:
df_species = df_species.iloc[: , :-1]
df_species.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


In [ ]:
df_parks.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [ ]:
print("df_wildlife shape", df_wildlife.shape)
print("df_species shape", df_species.shape)
print("df_parks shape", df_parks.shape)

df_wildlife shape (67161, 16)
df_species shape (119248, 14)
df_parks shape (56, 6)


# Data Clean


## Wildlife 

In [ ]:
## DROP SOURCE(how animal/plant was brought onto the market) AND Purpose
df_wildlife = df_wildlife.iloc[:,:-2]

### merge importer quantity and outputer quantity, and merge unit to the quantity
 If not specify, quantity is an unit


In [ ]:
df_wildlife.loc[df_wildlife['Importer reported quantity'].notnull() , 'quantity'] = df_wildlife['Importer reported quantity']

In [ ]:
df_wildlife.loc[df_wildlife['Exporter reported quantity'].notnull() , 'quantity'] = df_wildlife['Exporter reported quantity']

In [ ]:
df_wildlife.drop(columns = ['Importer reported quantity', 'Exporter reported quantity'], inplace = True)

In [ ]:
df_wildlife.loc[df_wildlife['Unit'].notnull() , 'quantity'] = df_wildlife['quantity'].astype(int).astype(str) + df_wildlife['Unit']

In [ ]:
df_wildlife.drop(columns = ['Unit'], inplace = True)

In [ ]:
df_wildlife.isna().sum()

In [ ]:
##too many missing value in class and origin, so we drop the columns
df_wildlife.drop(columns=['Class', 'Origin'], inplace = True)

In [ ]:
df_wildlife = df_wildlife.dropna()

In [ ]:
df_wildlife.to_csv("wildlifeTrade_clean.csv", index=True)
from google.colab import files
files.download('wildlifeTrade_clean.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Species

In [ ]:
df_species.isna().sum()

Species ID                  0
Park Name                   0
Category                    0
Order                    1472
Family                   1512
Scientific Name             0
Common Names                0
Record Status               0
Occurrence              20142
Nativeness              25045
Abundance               42942
Seasonality             99091
Conservation Status    114530
dtype: int64

In [ ]:
## seasonality's descrption is:
## When the species can be found in the park. Blank if the species is found there year-round.
## so we fill nan in seasonality by "all year"
df_species['Seasonality'].fillna('all year', inplace = True)

In [ ]:
df_species['Seasonality'] = df_species['Seasonality'].str.lower()

In [ ]:
#fill conservation status, abundance, nativeness, occurrence
df_species['Conservation Status'].fillna('not evaluated', inplace = True)
df_species['Abundance'].fillna('unknown', inplace = True)
df_species['Nativeness'].fillna('unknown', inplace = True)
df_species['Occurrence'].fillna('unknown', inplace = True)

In [ ]:
df_species.dropna(subset = ["Order", "Family"], inplace = True)

In [ ]:
df_species.to_csv("species_clean.csv", index=True)
from google.colab import files
files.download('species_clean.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Fire

In [ ]:
cd /content/drive/Shareddrives/550-Project

/content/drive/Shareddrives/550-Project


In [ ]:
df_fires = pd.read_csv('Fires.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_fires.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63.0,63.0,Plumas,NaN
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61.0,61.0,Placer,NaN
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17.0,17.0,El Dorado,NaN
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3.0,3.0,Alpine,NaN
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3.0,3.0,Alpine,NaN


In [ ]:
#drop shape
df_fires.drop(columns= ['Shape'], inplace = True)

In [ ]:
df_fires.drop(columns=['LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','ICS_209_INCIDENT_NUMBER','ICS_209_NAME','MTBS_ID', \
                       'MTBS_FIRE_NAME','COMPLEX_NAME','DISCOVERY_TIME'], inplace = True)

In [ ]:
df_fires.drop(columns = ['FIRE_NAME','CONT_DATE','CONT_DOY', 'CONT_TIME'], inplace = True)

In [ ]:
df_fires.isna().sum()

OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
COUNTY                        678148
FIPS_CODE                     678148
FIPS_NAME                     678148
dtype: int64

In [ ]:
df_fires.drop(columns = ['COUNTY'], inplace = True)

In [ ]:
df_fires.drop(columns = ['FIPS_CODE', 'FIPS_NAME'], inplace = True)

In [ ]:
df_fires.to_csv("fires_clean.csv", index=True)
from google.colab import files
files.download('fires_clean.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>